# IFT870 - Forage de données - TP2

Membres de l'équipe :
- Joë Larue
- Aurélien Vauthier

In [99]:
# import libraries
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# setup notebook parameters
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# extract data
package = pd.read_csv("Package2.csv", delimiter=';', encoding="latin1")
product = pd.read_csv("Product2.csv", delimiter=';', encoding="latin1")

## 1. Auscultez les données et présentez un résumé de votre auscultation

In [18]:
print(f"Affichons un aperçu des premières observations de package (taille : {package.shape} :")
package.head()

Affichons un aperçu des premières observations de package (taille : (173887, 8) :


,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N


In [19]:
print(f"Affichons un aperçu des premières observations de product (taille : {product.shape} :")
product.head()

Affichons un aperçu des premières observations de product (taille : (93238, 20) :


,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0


### Colonnes en double
- package contient plus de lignes que product
- PRODUCTID : données manquantes (chez product uniquement) & fusion entre product NDC et SPL document ID & duplicatas (ex 30901-30904)
- PRODUCTNDC : concaténation du code de l'étiqueteuse et du produit & données manquantes chez package (possibilité de récupération via PRODUCTID) & données abérantes chez product (dates ex: 320 (product) et 36655 (package))
- STARTMARKETINGDATE : date au format (AAAAMMJJ) & valeur bruité dans package (ex 33 avec 2997 au lieu de 1997)
- ENDMARKETINGDATE : date au format (AAAAMMJJ), valeurs abérantes dans product (ex 29505 avec année 3031)
- NDC_EXCLUDE_FLAG : uniquement la valeur 'N' car les autres produit (marqué 'Y', 'E' ou 'I', sont dans un autre fichier)

#### Colonnes de product seulement
- PRODUCTTYPENAME : pas de données manquantes
- PROPRIETARYNAME : valeurs abérantes (similaires ex 93068-93072, infos supplémentaires ex 430, erreur ex 7414)
- PROPRIETARYNAMESUFFIX : valeurs manquantes & optionnel & manque de standard
- NONPROPRIETARYNAME : valeurs abérantes (similaires : ',' et 'and', majuscules / minuscules)
- DOSAGEFORMNAME : liste d'attribut représentant l'apparence
- ROUTENAME : valeurs manquantes & liste d'attribut représentant les voies d'administrations
- MARKETINGCATEGORYNAME : en lien avec APPLICATIONNUMBER & valeurs similaires (ex UNAPPROVED DRUG OTHER, UNAPPROVED MDECIAL GAS...)
- APPLICATIONNUMBER : soit numéro NDA/ANDA/BLA ou "part X" pour les catégories OTC ou manquante pour les "UNAPPROVED" 
- LABELERNAME : valeurs abérantes ("10") & valeurs similaires (ex 25788-25790)
- SUBSTANCENAME : listes de substances & valeurs manquantes
- ACTIVE_NUMERATOR_STRENGTH (aka STRENGTHNUMBER) : quantité pour chaque substance (cf SUBSTANCENAME) & valeurs manquantes (lorsque substances manquantes)
- ACTIVE_INGRED_UNIT (aka STRENGTHUNIT) : liste des unité de mesure pour chaque quantité (cf ACTIVE_NUMERATOR_STRENGTH) & valeurs manquantes (lorsque substances manquantes)
- PHARM_CLASSES : valeurs manquantes & liste de classe
- DEASCHEDULE : valeurs manquantes & énumeration {CI, CII, CIII, CIV, CV} & CI non présente (peut-être null = CI ?) 
- LISTING_RECORD_CERTIFIED_THROUGH : valeurs manquantes & date d'expiration du statut à jour de la ligne (au format AAAAMMJJ) & peut-être redondant avec NDC_EXCLUDE_FLAG ?

Les colonnes SUBSTANCENAME, ACTIVE_NUMERATOR_STRENGTH et ACTIVE_INGRED_UNIT forme un trio de listes qui ont la même taille pour chaque médicaments (ou sont toutes les trois manquantes)

#### Colonnes de package seulement
- NDCPACKAGECODE : valeur de PRODUCTNDC + un code correspondant au package & valeurs abérantes (date ex 41903)
- PACKAGEDESCRIPTION : zone de description contenant un duplicata du NDCPACKAGECODE & décrit la quantité de médicament dans chaque package et sous-package
- SAMPLE_PACKAGE : 'Y' quand un package est à distribuer comme en échantillon.

Il y a des ProductID manquant dans le fichier `product`

## 3. Détectez et corrigez les incohérences entre des valeurs d’attributs dans les deux tables

In [35]:
### Checking NDC related columns TODO

# def checkAndRepairNDC(line):
#     validFormat = r"^[0-9]{4,5}-[0-9]{4}$"
#     
#     for 
#     
#     if not re.match(validFormat, NDC):
#         NDC = np.NaN
#     if not re.match(validFormat, relatedVal):
#         relatedVal = np.NaN
# 
#     if NDC == relatedVal == np.NaN:
#         print("HAAAAAA !")
# 
#     return NDC, relatedVal
#         
# product[["PRODUCTID", "PRODUCTNDC"]].apply(checkAndRepairNDC, axis=1)

In [49]:
### Checking STARTMARKETINGDATE is before ENDMARKETINGDATE

# Convert to date
product["STARTMARKETINGDATE"] = pd.to_datetime(product["STARTMARKETINGDATE"], format="%Y%m%d")
product["ENDMARKETINGDATE"] = pd.to_datetime(product["ENDMARKETINGDATE"], format="%Y%m%d", errors="coerce")
package["STARTMARKETINGDATE"] = pd.to_datetime(package["STARTMARKETINGDATE"], format="%Y%m%d", errors="coerce")
package["ENDMARKETINGDATE"] = pd.to_datetime(package["ENDMARKETINGDATE"], format="%Y%m%d", errors="coerce")

productInvalidDateMask = product["STARTMARKETINGDATE"] > product["ENDMARKETINGDATE"]
packageInvalidDateMask = package["STARTMARKETINGDATE"] > package["ENDMARKETINGDATE"]

if productInvalidDateMask.sum() > 0:
    print(f"Il y a {productInvalidDateMask.sum()} incohérence(s) entre STARTMARKETINGDATE et ENDMARKETINGDATE dans product.")
else:
    print("Il n'y a pas d'incohérence entre STARTMARKETINGDATE et ENDMARKETINGDATE dans product.")
if packageInvalidDateMask.sum() > 0:
    print(f"Il y a {packageInvalidDateMask.sum()} incohérence(s) entre STARTMARKETINGDATE et ENDMARKETINGDATE dans package.")
else:
    print("Il n'y a pas d'incohérence entre STARTMARKETINGDATE et ENDMARKETINGDATE dans package.")

Il n'y a pas d'incohérence entre STARTMARKETINGDATE et ENDMARKETINGDATE dans product.
Il n'y a pas d'incohérence entre STARTMARKETINGDATE et ENDMARKETINGDATE dans package.


In [50]:
### Checking NDC_EXCLUDE_FLAG

productInvalidExcludeFlag = product["NDC_EXCLUDE_FLAG"] != 'N'
packageInvalidExcludeFlag = package["NDC_EXCLUDE_FLAG"] != 'N'

if productInvalidExcludeFlag.sum() > 0:
    print(f"Il y a {productInvalidExcludeFlag.sum()} marquée(s) comme dépassée(s) (non à jour) dans product.")
else:
    print("Il n'y a pas de donnée marquée comme dépassée (non à jour) dans product.")
if packageInvalidExcludeFlag.sum() > 0:
    print(f"Il y a {packageInvalidExcludeFlag.sum()} marquée(s) comme dépassée(s) (non à jour) dans package.")
else:
    print("Il n'y a pas de donnée marquée comme dépassée (non à jour) dans package.")

Il n'y a pas de donnée marquée comme dépassée (non à jour) dans product.
Il n'y a pas de donnée marquée comme dépassée (non à jour) dans package.


In [95]:
# Define a generic function for checking the rightness of a column's value according to a dict and allowing replacement
def checkColumnFromDict(row, columnName=None, AcceptableTerms=None):    
    # check param
    if columnName is None or AcceptableTerms is None:
        return np.NaN
    if row[columnName] is np.NaN:
        return np.NaN
    
    # split for multiple value columns
    values = [val.strip() for val in row[columnName].split(';')]
    
    for i, value in enumerate(values):
        if isinstance(AcceptableTerms, dict):
            values[i] = AcceptableTerms.get(value, np.NaN)
        elif not value in AcceptableTerms:
            values[i] = np.NaN
    
    if np.NaN in values:
        return np.NaN
    return '; '.join(values)

In [71]:
### Check the DOSAGEFORMNAME values in product

# list found from https://www.fda.gov/industry/structured-product-labeling-resources/dosage-forms
# the different sub-dosage forms are removed to simplify the column values
DosageSPLAcceptableTerm = {"AEROSOL": "AEROSOL","AEROSOL, FOAM": "AEROSOL","AEROSOL, METERED": "AEROSOL","AEROSOL, POWDER": "AEROSOL","AEROSOL, SPRAY": "AEROSOL","BAR": "BAR","BAR, CHEWABLE": "BAR","BEAD": "BEAD","CAPSULE": "CAPSULE","CAPSULE, COATED": "CAPSULE","CAPSULE, COATED PELLETS": "CAPSULE","CAPSULE, COATED, EXTENDED RELEASE": "CAPSULE","CAPSULE, DELAYED RELEASE": "CAPSULE","CAPSULE, DELAYED RELEASE PELLETS": "CAPSULE","CAPSULE, EXTENDED RELEASE": "CAPSULE","CAPSULE, FILM COATED, EXTENDED RELEASE": "CAPSULE","CAPSULE, GELATIN COATED": "CAPSULE","CAPSULE, LIQUID FILLED": "CAPSULE","CELLULAR SHEET": "CELLULAR SHEET","CHEWABLE GEL": "CHEWABLE GEL","CLOTH": "CLOTH","CONCENTRATE": "CONCENTRATE","CREAM": "CREAM","CREAM, AUGMENTED": "CREAM","CRYSTAL": "CRYSTAL","DISC": "DISC","DOUCHE": "DOUCHE","DRESSING": "DRESSING","ELIXIR": "ELIXIR","EMULSION": "EMULSION","ENEMA": "ENEMA","EXTRACT": "EXTRACT","FIBER, EXTENDED RELEASE": "FIBER","FILM": "FILM","FILM, EXTENDED RELEASE": "FILM","FILM, SOLUBLE": "FILM","FOR SOLUTION": "FOR SOLUTION","FOR SUSPENSION": "FOR SUSPENSION","FOR SUSPENSION, EXTENDED RELEASE": "FOR SUSPENSION","GAS": "GAS","GEL": "GEL","GEL, DENTIFRICE": "GEL","GEL, METERED": "GEL","GLOBULE": "GLOBULE","GRANULE": "GRANULE","GRANULE, DELAYED RELEASE": "GRANULE","GRANULE, EFFERVESCENT": "GRANULE","GRANULE, FOR SOLUTION": "GRANULE","GRANULE, FOR SUSPENSION": "GRANULE","GRANULE, FOR SUSPENSION, EXTENDED RELEASE": "GRANULE","GUM": "GUM","GUM, CHEWING": "GUM","IMPLANT": "IMPLANT","INHALANT": "INHALANT","INJECTABLE FOAM": "INJECTABLE FOAM","INJECTABLE": "INJECTABLE","INJECTABLE, LIPOSOMAL": "INJECTABLE","INJECTION": "INJECTION","INJECTION, EMULSION": "INJECTION","INJECTION, LIPID COMPLEX": "INJECTION","INJECTION, POWDER, FOR SOLUTION": "INJECTION","INJECTION, POWDER, FOR SUSPENSION": "INJECTION","INJECTION, POWDER, FOR SUSPENSION, EXTENDED RELEASE": "INJECTION","INJECTION, POWDER, LYOPHILIZED, FOR LIPOSOMAL SUSPENSION": "INJECTION","INJECTION, POWDER, LYOPHILIZED, FOR SOLUTION": "INJECTION","INJECTION, POWDER, LYOPHILIZED, FOR SUSPENSION": "INJECTION","INJECTION, POWDER, LYOPHILIZED, FOR SUSPENSION, EXTENDED RELEASE": "INJECTION","INJECTION, SOLUTION": "INJECTION","INJECTION, SOLUTION, CONCENTRATE": "INJECTION","INJECTION, SUSPENSION": "INJECTION","INJECTION, SUSPENSION, EXTENDED RELEASE": "INJECTION","INJECTION, SUSPENSION, LIPOSOMAL": "INJECTION","INJECTION, SUSPENSION, SONICATED": "INJECTION","INSERT": "INSERT","INSERT, EXTENDED RELEASE": "INSERT","INTRAUTERINE DEVICE": "INTRAUTERINE DEVICE","IRRIGANT": "IRRIGANT","JELLY": "JELLY","KIT": "KIT","LINIMENT": "LINIMENT","LIPSTICK": "LIPSTICK","LIQUID": "LIQUID","LIQUID, EXTENDED RELEASE": "LIQUID","LOTION": "LOTION","LOTION, AUGMENTED": "LOTION","LOTION/SHAMPOO": "LOTION/SHAMPOO","LOZENGE": "LOZENGE","MOUTHWASH": "MOUTHWASH","NOT APPLICABLE": "NOT APPLICABLE","OIL": "OIL","OINTMENT": "OINTMENT","OINTMENT, AUGMENTED": "OINTMENT","PASTE": "PASTE","PASTE, DENTIFRICE": "PASTE","PASTILLE": "PASTILLE","PATCH": "PATCH","PATCH, EXTENDED RELEASE": "PATCH","PATCH, EXTENDED RELEASE, ELECTRICALLY CONTROLLED": "PATCH","PELLET": "PELLET","PELLET, IMPLANTABLE": "PELLET","PELLETS, COATED, EXTENDED RELEASE": "PELLETS","PILL": "PILL","PLASTER": "PLASTER","POULTICE": "POULTICE","POWDER": "POWDER","POWDER, DENTIFRICE": "POWDER","POWDER, FOR SOLUTION": "POWDER","POWDER, FOR SUSPENSION": "POWDER","POWDER, METERED": "POWDER","RING": "RING","RINSE": "RINSE","SALVE": "SALVE","SHAMPOO": "SHAMPOO","SHAMPOO, SUSPENSION": "SHAMPOO","SOAP": "SOAP","SOLUTION": "SOLUTION","SOLUTION, CONCENTRATE": "SOLUTION","SOLUTION, FOR SLUSH": "SOLUTION","SOLUTION, GEL FORMING / DROPS": "SOLUTION","SOLUTION, GEL FORMING, EXTENDED RELEASE": "SOLUTION","SOLUTION/ DROPS": "SOLUTION/ DROPS","SPONGE": "SPONGE","SPRAY": "SPRAY","SPRAY, METERED": "SPRAY","SPRAY, SUSPENSION": "SPRAY","STICK": "STICK","STRIP": "STRIP","SUPPOSITORY": "SUPPOSITORY","SUPPOSITORY, EXTENDED RELEASE": "SUPPOSITORY","SUSPENSION": "SUSPENSION","SUSPENSION, EXTENDED RELEASE": "SUSPENSION","SUSPENSION/ DROPS": "SUSPENSION/ DROPS","SWAB": "SWAB","SYRUP": "SYRUP","SYSTEM": "SYSTEM","TABLET": "TABLET","TABLET, CHEWABLE": "TABLET","TABLET, CHEWABLE, EXTENDED RELEASE": "TABLET","TABLET, COATED": "TABLET","TABLET, COATED PARTICLES": "TABLET","TABLET, DELAYED RELEASE": "TABLET","TABLET, DELAYED RELEASE PARTICLES": "TABLET","TABLET, EFFERVESCENT": "TABLET","TABLET, EXTENDED RELEASE": "TABLET","TABLET, FILM COATED": "TABLET","TABLET, FILM COATED, EXTENDED RELEASE": "TABLET","TABLET, FOR SOLUTION": "TABLET","TABLET, FOR SUSPENSION": "TABLET","TABLET, MULTILAYER": "TABLET","TABLET, MULTILAYER, EXTENDED RELEASE": "TABLET","TABLET, ORALLY DISINTEGRATING": "TABLET","TABLET, ORALLY DISINTEGRATING, DELAYED RELEASE": "TABLET","TABLET, SOLUBLE": "TABLET","TABLET, SUGAR COATED": "TABLET","TABLET WITH SENSOR": "TABLET WITH SENSOR","TAMPON": "TAMPON","TAPE": "TAPE","TINCTURE": "TINCTURE","TROCHE": "TROCHE","WAFER": "WAFER"}

product["DOSAGEFORMNAME"] = product.apply(checkColumnFromDict, axis=1, columnName="DOSAGEFORMNAME", AcceptableTerms=DosageSPLAcceptableTerm)
nbNotAccepted = product["DOSAGEFORMNAME"].isna().sum()

if nbNotAccepted > 0:
    print(f"Il y a {nbNotAccepted} forme(s) de dosage(s) invalide(s).")
else:
    print("Il n'y a aucune forme de dosage invalide.")

Il n'y a aucune forme de dosage invalide.


In [100]:
### Check the DOSAGEFORMNAME values in product

# list found from https://www.fda.gov/industry/structured-product-labeling-resources/route-administration
routeSPLAcceptableTerm = ["AURICULAR (OTIC)","BUCCAL","CONJUNCTIVAL","CUTANEOUS","DENTAL","ELECTRO-OSMOSIS","ENDOCERVICAL","ENDOSINUSIAL","ENDOTRACHEAL","ENTERAL","EPIDURAL","EXTRA-AMNIOTIC","EXTRACORPOREAL","HEMODIALYSIS","INFILTRATION","INTERSTITIAL","INTRA-ABDOMINAL","INTRA-AMNIOTIC","INTRA-ARTERIAL","INTRA-ARTICULAR","INTRABILIARY","INTRABRONCHIAL","INTRABURSAL","INTRACANALICULAR","INTRACARDIAC","INTRACARTILAGINOUS","INTRACAUDAL","INTRACAVERNOUS","INTRACAVITARY","INTRACEREBRAL","INTRACISTERNAL","INTRACORNEAL","INTRACORONAL, DENTAL","INTRACORONARY","INTRACORPORUS CAVERNOSUM","INTRACRANIAL","INTRADERMAL","INTRADISCAL","INTRADUCTAL","INTRADUODENAL","INTRADURAL","INTRAEPICARDIAL","INTRAEPIDERMAL","INTRAESOPHAGEAL","INTRAGASTRIC","INTRAGINGIVAL","INTRAHEPATIC","INTRAILEAL","INTRALESIONAL","INTRALINGUAL","INTRALUMINAL","INTRALYMPHATIC","INTRAMAMMARY","INTRAMEDULLARY","INTRAMENINGEAL","INTRAMUSCULAR","INTRANODAL","INTRAOCULAR","INTRAOMENTUM","INTRAOVARIAN","INTRAPERICARDIAL","INTRAPERITONEAL","INTRAPLEURAL","INTRAPROSTATIC","INTRAPULMONARY","INTRARUMINAL","INTRASINAL","INTRASPINAL","INTRASYNOVIAL","INTRATENDINOUS","INTRATESTICULAR","INTRATHECAL","INTRATHORACIC","INTRATUBULAR","INTRATUMOR","INTRATYMPANIC","INTRAUTERINE","INTRAVASCULAR","INTRAVENOUS","INTRAVENTRICULAR","INTRAVESICAL","INTRAVITREAL","IONTOPHORESIS","IRRIGATION","LARYNGEAL","NASAL","NASOGASTRIC","NOT APPLICABLE","OCCLUSIVE DRESSING TECHNIQUE","OPHTHALMIC","ORAL","OROPHARYNGEAL","PARENTERAL","PERCUTANEOUS","PERIARTICULAR","PERIDURAL","PERINEURAL","PERIODONTAL","RECTAL","RESPIRATORY (INHALATION)","RETROBULBAR","SOFT TISSUE","SUBARACHNOID","SUBCONJUNCTIVAL","SUBCUTANEOUS","SUBGINGIVAL","SUBLINGUAL","SUBMUCOSAL","SUBRETINAL","TOPICAL","TRANSDERMAL","TRANSENDOCARDIAL","TRANSMUCOSAL","TRANSPLACENTAL","TRANSTRACHEAL","TRANSTYMPANIC","URETERAL","URETHRAL","VAGINAL"]

product["ROUTENAME"] = product.apply(checkColumnFromDict, axis=1, columnName="ROUTENAME", AcceptableTerms=routeSPLAcceptableTerm)
nbNotAccepted = product["ROUTENAME"].isna().sum() - 1932 # remove the number of initial missing values

if nbNotAccepted > 0:
    print(f"Il y a {nbNotAccepted} voie(s) d'administration(s) invalide(s).")
else:
    print("Il n'y a aucune voie d'administration invalide.")

Il n'y a aucune voie d'administration invalide.


In [ ]:
product["LISTING_RECORD_CERTIFIED_THROUGH"] = pd.to_datetime(product["LISTING_RECORD_CERTIFIED_THROUGH"], format="%Y%m%d")

## 4. Complétez au maximum les données manquantes dans les deux tables

## 5. Détectez et retirez les objets dupliqués dans les deux tables

## 6. Intégrez les deux tables et nettoyez le résultat

## 7. Proposez un nouvel ensemble d’attributs (représentation) qui élimine la redondance des informations dans les valeurs des attributs

## 8. À partir de la nouvelle représentation, proposez un ensemble d’attributs à utiliser pour prédire toutes les classes pharmaceutiques d’un médicament

## 9. Appliquez un modèle de classification pour prédire les classes pharmaceutiques des médicaments pour lesquels l’information est manquante

## 10. Évaluez un échantillon de vos résultats à l’aide de connaissances d’experts